# CREATE FLAG PARAMETER

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag = dbutils.widgets.get('incremental_flag')
print(type(incremental_flag))

# CREATING DIMENION MODEL

# FETCH RELATIVE COLUMNS

In [0]:
df_src = spark.sql("""
    SELECT DISTINCT MODEL_ID AS Model_ID, MODEL_CATEGORY 
    FROM PARQUET.`abfss://silver@carsneelabhdatalake.dfs.core.windows.net/carsales`
""")


In [0]:
df_src.display()

# DIM MODEL SINK INITIAL AND INCREMENTAL

In [0]:
if  spark.catalog.tableExists('carsproject.gold.dim_model'):
    df_sink = spark.sql("""
        SELECT 1 AS dim_model_key, MODEL_ID, MODEL_CATEGORY 
        FROM PARQUET.`abfss://silver@carsneelabhdatalake.dfs.core.windows.net/carsales`
    """)
else:
    df_sink = spark.sql("""
        SELECT 1 AS dim_model_key, MODEL_ID, MODEL_CATEGORY 
        FROM PARQUET.`abfss://silver@carsneelabhdatalake.dfs.core.windows.net/carsales`
        WHERE 1 = 0
    """)


# FILTERING NEW RECORDS AND OLD RECORDS

In [0]:
df_filter = df_src.join(df_sink, df_src['Model_ID'] == df_sink['MODEL_ID'], 'left').select(df_src['MODEL_ID'], df_src['Model_Category'],df_sink['dim_model_key'])


In [0]:
df_filter.display()